# HASH Table
## Что такое Hash Table?
Хеш таблица — это структура данных, которая хранит пары ключ-значение. Ключ в нашем случае это строка, а значение — это адрес в памяти, где хранится значение. По ключу можно узнать bucket, в котором хранится значение, однако при коллизии приходится дополнительно идти по списку bucketа, чтобы найти нужное значение. 


![hashTable_normalSituation](imgs/hashTable_normalSituation.svg)

Коллизия — это ситуация, когда два ключа имеют одинаковый хэш. В таком случае два слова могут попасть в один bucket. Пэтому bucket — это список, в котором хранятся все значения, имеющие одинаковый хэш. У меня bucket - это двусвязный список. Можно было сделать односвязный список, но в скорости удаления элемента он проигрывает.

![hashTable_collision](imgs/hashTable_collision.svg)

## Оптимизации
### 1. Переключение с `-O0` на `-O2`
<style>
.highlight-box {
    color: #2ecc71; /* Зелёный цвет текста */
    font-weight: bold; /* Жирный шрифт */
}
</style>

<style>
.code-container {
    display: flex; /* Располагаем элементы в строке */
    justify-content: space-between; /* Распределяем пространство между колонками */
    gap: 0px; /* Отступ между колонками */
    font-size: 12px;
}

.code-column {
    width: 48%; /* Ширина каждой колонки */
}
</style>


Самая простая и очевидная оптимизация это включить оптимизацию компилятора `-O2`. Это позволяет компилятору использовать более эффективные алгоритмы.  
В таком случае получем следующую информацию сравнения:
<details>
<summary>Click to show comparison</summary>  

Первое число - `-O2`, второе число - `-O0`  

**Time**  

`Elapsed Time`:	0.393s - 0.380s = 0.013s  
`CPU Time`:	0.345s - 0.359s = -0.014s  

**Hardware Events**  

| Hardware Event Type | Hardware Event Count  | Hardware Event Sample Count	| Events Per Sample | Precise |
|---------------------|-----------------------|-----------------------------|-------------------|---------|
|cycles	| 828,806,417 - 861,866,547 = -33,060,130 | 1,589 - 1,536 = 53 | 4000 | Not changed, False |

**Вкладка Caller/Calle**  

![compare-O2and-O0](imgs/compare-O2and-O0.png)

</details>  

<div class="highlight-box">Процессорное время уменьшилось на 3.3 %, что достаточно хороший результат.</div>

### 2. Добавление SIMD instructions
Из предыдущего пункта видно, что много тактов процессора уходит на вызов функции `strcmp`, которая находится в `searchHT`. Это можно исправить, если использовать SIMD instructions. Условимся на том, что каждое слова будет не более 32 байт. В таком случае можно обрабатывать сразу по четыре слова, используя векторы `_mm256i`.  

<div class="code-container">
    <div class="code-column">

**Старая версия:**  
```c
    int searchHT(HashTable* table, const unsigned char* word)
    {
        assert(table);
        assert(word);
        
        size_t index = hashFunction(word);
        Node* current = table->buckets[index].head;
        while (current != NULL) 
        {
            if (strcmp(current->word, (const char*)word) == 0) 
            {
                return 1;
            }
            current = current->next;
        }
        return 0;
    }
```
</div> <div class="code-column">

**Новая версия:**  
```c
    int searchHT(HashTable* table, const unsigned char* word) 
    {
        assert(table);
        assert(word);

        size_t index = hashFunction(word);
        Node* current = table->buckets[index].head;

        __m256i word_vec = _mm256_loadu_si256((const __m256i*)word);
        
        uint32_t word_first_4 = 0;
        uint32_t current_first_4 = 0;

        memcpy(&word_first_4, word, sizeof(uint32_t));

        while (current != NULL) 
        {
            memcpy(&current_first_4, current->word, sizeof(uint32_t));
            if (word_first_4 == current_first_4) 
            {
                __m256i current_vec = _mm256_loadu_si256((const __m256i*)current->word);
                
                __m256i cmp_result = _mm256_cmpeq_epi8(word_vec, current_vec);
                
                int mask = _mm256_movemask_epi8(cmp_result);

                if (mask == 0xFFFFFFFF) {
                    return 1;
                }
            }

            current = current->next;
        }
        return 0;
    }
```
</div> </div>
В таком случае получем следующую информацию сравнения:
<details>
<summary>Click to show comparison</summary>  

Первое число - `-O2 with AVX`, второе число - `-O2 no AVX`  

**Time**  

`Elapsed Time`:	0.393s - 0.380s = 0.013s  
`CPU Time`:	0.345s - 0.359s = -0.014s  

**Hardware Events**  

| Hardware Event Type | Hardware Event Count  | Hardware Event Sample Count	| Events Per Sample | Precise |
|---------------------|-----------------------|-----------------------------|-------------------|---------|
|cycles	| 828,806,417 - 861,866,547 = -33,060,130 | 1,589 - 1,536 = 53 | 4000 | Not changed, False |

**Вкладка Caller/Calle**  

![compare-O2and-O0](imgs/compare-O2and-O0.png)

</details>  

<div class="highlight-box">Процессорное время уменьшилось на 3.3 %, что достаточно хороший результат.</div>